# 실행 가능

- 저자 : [Hyeyeoon] (https://github.com/hyeyeoon)
-동료 검토 : [hong-seongmin] (https://github.com/hong-seongmin), [Wooseok Jeong] (https://github.com/jeong-wooseok)
- 교정 : [q011] (https://github.com/q0211)
-이것은 [Langchain Open Tutorial]의 일부입니다.

[!
[! [열린github] (https://img.shields.io/badge/open%20in%20github-181717?style=flat-square&logo=github&logocolor=white)]] (https://github.com/langchain-opentutorial/langchain-opentutorial/blob/main/01-basic/08-runnable.ipynb)

## 개요

Langchain의````runnable '' '객체는 체인, 병렬 실행 및 데이터 변환을 활성화하여 워크 플로 설계에 대한 모듈 식적이고 유연한 접근 방식을 제공합니다.이 유틸리티는 최소한의 코드 오버 헤드로 구조화 된 입력 및 출력을 효율적으로 처리 할 수 있습니다.

주요 구성 요소는 다음과 같습니다.

- **`````````````` "벨라 ''** : 람다 함수를 통해 사용자 정의 로직을 적용 할 수있는 가벼운 유틸리티. 동적 및 빠른 데이터 변환에 이상적입니다.
- **````````````````````````````````````````` "
- **`````````````** : 사전 또는 튜플과 같은 구조화 된 데이터에서 특정 키나 지수를 효율적으로 추출하기위한 Python`````운영자 ''모듈 유틸리티.

이러한 도구를 결합하여 다음과 같은 강력한 워크 플로를 구축 할 수 있습니다.

-````````````. "를 사용하여 특정 데이터 요소를 추출하고 처리합니다.
-````runnablelambda '' '로 사용자 정의 변환을 수행합니다.
-End-to-End Data Pipelines```````````` ''체인을 생성합니다.

이러한 구성 요소를 활용하여 사용자는 기계 학습 및 데이터 처리 워크 플로를위한 확장 가능하고 재사용 가능한 파이프 라인을 설계 할 수 있습니다.


### 목차

- [개요] (#개요)
- [환경 설정] (#Environment-Setup)
-[runnablePassthrough를 사용한 효율적인 데이터 처리] (#효율적인 데이터 처리와 함께 실행 가능한 passplasphrough)
-[runnableparallel을 사용한 효율적인 병렬 실행] (#효율적인 parallel-execution with Runnable Parallel)
-[runnablelambda를 사용한 동적 처리] (#dynamic processing with-runablelambda)
-[ItemGetter를 사용하여 특정 키 추출] (#추출 특이 적 주류-itemgetter)

### 참조

- [langchain 문서 : runnable] (https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.runnable.html)
- [langchain 문서] (https://python.langchain.com/docs/how_to/lcel_cheatsheet/)
- [Python 연산자 모듈 : itemgetter] (https://docs.python.org/3/library/operator.html#operator.itemgetter)

---

## 환경 설정

환경을 설정하십시오.자세한 내용은 [환경 설정] (https://wikidocs.net/257836)을 참조하십시오.

**[메모]**
-``Langchain-Opentutorial '' '는 사용하기 쉬운 환경 설정 세트, 유용한 기능 및 튜토리얼을위한 유틸리티를 제공하는 패키지입니다.
-자세한 내용은 [``````langchain-opentutorial``] (https://github.com/langchain-opentutorial/langchain-opentutorial-pypi)를 확인할 수 있습니다.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [2]:
#필요한 패키지를 설치하십시오
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

`````.env '' '````open_api_key````````Guste를로드 할 수도 있습니다.

In [3]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [5]:
#로컬 환경 변수를 설정합니다
from langchain_opentutorial import set_env

set_env(
    {
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "05-Runnable",
    }
)

Environment variables have been set successfully.


## RunnablePassthrough를 사용한 효율적인 데이터 처리

````runnablepassthrough '' '’는 입력 데이터를 변경하지 않거나 추가 속성으로이를 향상시켜 데이터 처리 워크 플로우를 간소화하도록 설계된 유틸리티입니다.유연성은 최소한의 변환 또는 선택적 확대가 필요한 파이프 라인의 데이터를 처리하는 데 유용한 도구입니다.

1. ** 간단한 데이터 전달 **

- 원시 데이터를 기록하거나 다운 스트림 시스템에 전달하는 등 변환이 필요하지 않은 시나리오에 적합합니다.

2. ** 동적 데이터 확대 **

- 머신 러닝 파이프 라인 또는 분석 시스템에 사용하기위한 입력 데이터에 메타 데이터 또는 컨텍스트를 추가 할 수 있습니다.

---
-````runnablepassthrough ''```````````````````````RunnablePassthrough (runnablePassthrough '’는 입력을 변경하지 않거나 추가 키를 추가 할 수 있습니다.
-``````````````runnablePassthrough ()``` ''가 자체적으로 호출되면, 단순히 입력을 가져 와서 그대로 전달합니다.
-``«runnablePassthrough.assign (...)`````` ''를 사용하여 호출 할 때 입력을 취하고 할당 함수에 제공된 추가 인수를 추가합니다.

### runnablepassthrough

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

#프롬프트와 LLM을 만듭니다
prompt = PromptTemplate.from_template("What is 10 times {num}?")
llm = ChatOpenAI(temperature=0)

#체인을 만듭니다
chain = prompt | llm

``«invoke ()``````` '``````'``Dictionary ''«이 입력 데이터가 있어야합니다.

In [7]:
#체인 실행 : 입력 dtype는 '사전'으로
chain.invoke({"num": 5})

AIMessage(content='10 times 5 is equal to 50.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-420bc9dc-12eb-4f7a-a2c4-8e521b3d952d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

그러나 Langchain 라이브러리로 업데이트되면 템플릿에 ** 하나의 변수 만 포함 된 경우 값을 직접 전달할 수도 있습니다.

In [8]:
#체인 실행 : 입력 값을 직접 입력하십시오
chain.invoke(5)

AIMessage(content='10 times 5 is equal to 50.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3723d11b-89e1-490c-8946-b724fbc2c46d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

다음은````runnablepassthrough '' '를 사용하는 예입니다.
````runnablepassthrough '' '`````````````````'```````````

1. ** 기본 작동 **
- 입력 값을 출력으로 직접 전달하는 간단한 통과 함수를 수행합니다.
-``«invoke ()````` ''메소드를 사용하여 독립적으로 실행할 수 있습니다

2. ** 사용 사례 **
- 수정없이 체인 단계를 통해 데이터를 전달해야 할 때 유용합니다.
- 다른 구성 요소와 결합하여 복잡한 데이터 파이프 라인을 구축 할 수 있습니다.
- 새 필드를 추가하면서 원래 입력을 보존해야 할 때 특히 도움이됩니다.

3. ** 입력 처리 **
- 사전 유형 입력을 허용합니다
- 단일 값도 처리 할 수 있습니다
- 체인 전체에 데이터 구조를 유지합니다

In [9]:
from langchain_core.runnables import RunnablePassthrough

#실행 가능
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

다음은````runnablepassthrough '' '를 사용하여 체인을 만드는 예입니다.

In [10]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

#DICT 값은 RunnablePassthrough ()로 업데이트되었습니다.
runnable_chain.invoke(10)

AIMessage(content='10 times 10 is equal to 100.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dffc0a69-0ee5-43b1-adae-03ee863d5a68-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

다음은````runnablepassthrough.assign ()```` '를 사용할 때 결과를 비교하는 것입니다.

In [11]:
RunnablePassthrough().invoke({"num": 1})

{'num': 1}

`````runnablepassthrough.assign ()````
- 입력의 키/값 쌍을 새로 할당 된 키/값 쌍과 결합합니다.

In [12]:
#입력 키 : NUM, 할당 키 : new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

## runnableparallel을 사용한 효율적인 병렬 실행

````runnableparallel`````````G 여러````runnable " ''객체를 동시에 실행하도록 설계된 유틸리티입니다. 병렬 처리가 필요한 워크 플로우를 간소화합니다.다른 구성 요소에 입력 데이터를 배포하고 결과를 수집하여 통합 출력으로 결합합니다.이 기능은 작업을 독립적이고 동시에 수행 할 수있는 워크 플로를 최적화하기위한 강력한 도구입니다.


1. ** 동시 실행 **
- 여러````````````` ''객체를 동시에 실행하여 병렬화 할 수있는 작업에 필요한 시간을 줄입니다.

2. ** 통합 출력 관리 **
- 모든 병렬 실행의 결과를 단일의 응집력있는 출력으로 결합하여 다운 스트림 처리를 단순화합니다.

3. ** 유연성 **
- 워크로드를 효율적으로 분배하여 다양한 입력 유형을 처리하고 복잡한 워크 플로우를 지원할 수 있습니다.

In [13]:
from langchain_core.runnables import RunnableParallel

#Runnable Parallel 인스턴스를 만듭니다.이 인스턴스를 사용하면 여러 실행 가능한 객체를 병렬로 실행할 수 있습니다.
runnable = RunnableParallel(
    #'통과 된'키워드 인수로 런닝 가능한 패스 스루 인스턴스를 전달하십시오.이것은 단순히 수정없이 입력 데이터를 통과시킵니다.
    passed=RunnablePassthrough(),
    #Lambda 함수 'Mult'를 할당하기 위해 runnablepassthrough.assign을 '추가'키워드 인수로 사용하십시오.
    #이 함수는 입력 사전의 'Num'키와 관련된 값에 3을 곱합니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    #Lambda 기능을 '수정 된'키워드 인수로 전달하십시오.
    #이 함수는 입력 사전의 'num'키와 관련된 값에 1을 추가합니다.
    modified=lambda x: x["num"] + 1,
)

#실행 가능한 인스턴스에서 Invoke 메소드를 호출하여 사전 { 'num': 1}을 입력으로 전달하십시오.
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

체인은 또한 런닝 가능한 파라럴에 적용될 수 있습니다.

In [14]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("What is the capital of {country}?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("What is the area of {country}?")
    | ChatOpenAI()
)

In [15]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("United States of America")

{'capital': AIMessage(content='The capital of the United States of America is Washington, D.C.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 17, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-29437a26-8661-4f15-a655-3b3ca6aa0e8c-0', usage_metadata={'input_tokens': 17, 'output_tokens': 15, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'area': AIMessage(content='The total land area of the United States of America is approximately 3.8 million square miles.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens

## runnablelambda를 사용한 동적 처리

``````runnablelambda '' ''는 개발자가 Lambda 함수를 사용하여 사용자 정의 데이터 변환 로직을 정의 할 수있는 유연한 유틸리티입니다.사용자 정의 처리 워크 플로의 빠르고 쉽게 구현할 수있게함으로써````runnableAmbda '' '는 맞춤형 데이터 파이프 라인의 생성을 단순화하면서 최소 설정 오버 헤드를 보장합니다.

1. ** 사용자 정의 가능한 데이터 변환 **
- 사용자는 Lambda 함수를 사용하여 입력 데이터를 변환하기위한 사용자 정의 로직을 정의 할 수 있으며 비교할 수없는 유연성을 제공합니다.

2. ** 경량과 간단한 **
- 광범위한 클래스 또는 기능 정의없이 임시 처리를 구현하는 간단한 방법을 제공합니다.

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime

def get_today(a):
    #오늘 날짜를 얻으십시오
    return datetime.today().strftime("%b-%d")

#오늘 날짜를 인쇄하십시오
get_today(None)

'Jan-04'

In [17]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

#프롬프트와 LLM을 만듭니다
prompt = PromptTemplate.from_template(
    "List {n} famous people whose birthday is on {today}. Include their date of birth."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

#체인을 만듭니다
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
#산출
print(chain.invoke(3))

Here are three famous people born on January 4:

1. **Isaac Newton** - Born on January 4, 1643 (according to the Gregorian calendar; December 25, 1642, in the Julian calendar), he was an English mathematician, physicist, astronomer, and author who is widely recognized as one of the most influential scientists of all time.

2. **Louis Braille** - Born on January 4, 1809, he was a French educator and inventor of a system of reading and writing for use by the blind or visually impaired, known as Braille.

3. **Michael Stipe** - Born on January 4, 1960, he is an American singer-songwriter and the lead vocalist of the alternative rock band R.E.M.


##````ItemGetter '' '를 사용하여 특정 키를 추출합니다

```````````````````ution '`````````````````Operator' '`````````````````````````'모듈은 다음과 같은 기능과 이점이 있습니다.

1. ** 핵심 기능 **
- 사전, 튜플 및 목록의 특정 키 또는 지수에서 값을 효율적으로 추출합니다.
- 여러 키나 지수를 동시에 추출 할 수 있습니다
- 기능 프로그래밍 스타일을 지원합니다

2. ** 성능 최적화 **
- 반복적 인 키 액세스 작업을위한 일반 인덱싱보다 효율적입니다.
- 최적화 된 메모리 사용
- 큰 데이터 세트를 처리 할 때 성능 장점

3. ** 랑 체인의 사용 **
- 체인 구성의 데이터 필터링
- 복잡한 입력 구조에서 선택적 추출
- 데이터 전처리를 위해 다른 런닝 가능한 객체와 결합합니다

In [19]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


#문장의 길이를 반환하는 함수.
def length_function(text):
    return len(text)


#두 문장 길이의 곱을 반환하는 함수.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


#_multiple_length_function을 사용하는 함수는 두 문장 길이의 제품을 반환합니다.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("What is {a} + {b}?")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [20]:
chain.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='5 + 25 = 30', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1cb2a062-52ba-4042-a4c1-a1eef155f6cc-0', usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})